In [4]:
from google.colab import auth
auth.authenticate_user()

import ee
ee.Authenticate()
ee.Initialize()

import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

import folium
print(folium.__version__)


import numpy as np
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, UpSampling2D
from keras.callbacks import EarlyStopping
from keras.models import Model
from keras.optimizers import Adam
import pandas as pd

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=E-CNfh-d-B04aB_RuIhSGjjORNXI7Cq69hmno3P69Tw&tc=VOoZeoPtix3UcWNUD3c87MY1-kDIvFn_wpsu2xUhLtA&cc=nvsgTfJ-MsxmVZce-RI3G9yko9Bg9HmQwBkB0Px3kJA

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk7DbrOBIvcJQzrEDCQRF2jvWdmU99annN51fyILFbQrYCf2UJj5O8g

Successfully saved authorization token.
2.11.0
0.14.0


# Read-in

In [ ]:
hansen_dataset = ee.Image('UMD/hansen/global_forest_change_2020_v1_8')

In [15]:
# Load Global Forest Change dataset
forestChange = ee.Image('UMD/hansen/global_forest_change_2020_v1_8')

# Define region of interest
roi = ee.Geometry.Point([-75.58, 6.41]).buffer(10000)

# Extract forest loss and gain pixels as separate binary masks
forestLoss = forestChange.select(['gain']).selfMask()
forestGain = forestChange.select(['loss']).selfMask()

# Add a constant value band to the binary masks
constant = ee.Image(1)
forestLoss = forestLoss.addBands(constant)
forestGain = forestGain.addBands(constant)

# Convert forest loss and gain masks to feature collections
forestLossFeatures = forestLoss.reduceToVectors(
    geometry=roi,
    geometryType='polygon',
    crs=forestChange.projection(),
    scale=30,
    eightConnected=True,
    labelProperty='class',
    reducer=ee.Reducer.count()
)

forestGainFeatures = forestGain.reduceToVectors(
    geometry=roi,
    geometryType='polygon',
    crs=forestChange.projection(),
    scale=30,
    eightConnected=True,
    labelProperty='class',
    reducer=ee.Reducer.count()
)

# Combine forest loss and gain features into a single feature collection
labeledData = forestLossFeatures.merge(forestGainFeatures)

# Print the first 10 features of the labeled data
print(labeledData.limit(10).getInfo())

{'type': 'FeatureCollection', 'columns': {'class': 'Short<0, 255>', 'count': 'Long<0, 4294967295>', 'system:index': 'String'}, 'features': [{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[-75.66651879886435, 6.428247725196414], [-75.6662493042791, 6.428247725196414], [-75.6662493042791, 6.428517219781654], [-75.66651879886435, 6.428517219781654], [-75.66651879886435, 6.428247725196414]]]}, 'id': '1_+387145+272998', 'properties': {'class': 1, 'count': 1}}, {'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[-75.66597980969388, 6.428517219781654], [-75.66571031510864, 6.428517219781654], [-75.66571031510864, 6.428786714366893], [-75.66597980969388, 6.428786714366893], [-75.66597980969388, 6.428517219781654]]]}, 'id': '1_+387147+272997', 'properties': {'class': 1, 'count': 1}}, {'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[-75.66301536925629, 6.439297003191086], [-75.66

In [16]:
labeledData

In [24]:
import pandas as pd
import time
from google.colab import auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


# Combine forest loss and gain features into a single feature collection
labeledData = forestLossFeatures.merge(forestGainFeatures)

# Export the labeled data to a CSV file in Google Drive
task = ee.batch.Export.table.toDrive(
    collection=labeledData,
    description='labeledData',
    folder='GEE_exports',
    fileFormat='CSV'
)
task.start()

# Wait for the export task to complete
while task.status()['state'] == 'RUNNING':
    print('Exporting labeledData...')
    time.sleep(5)
print('Export completed.')

# Download the exported CSV file from Google Drive
file_list = drive.ListFile({'q': "'GEE_exports' in parents and trashed=false"}).GetList()
file_name = 'labeledData.csv'
for file1 in file_list:
    if file1['title'] == file_name:
        file_id = file1['id']
        file = drive.CreateFile({'id': file_id})
        file.GetContentFile(file_name)
        break

# Load the CSV file into a Pandas dataframe
dataframe = pd.read_csv('labeledData.csv')


Export completed.


HttpError: ignored